# Mandatory Assignment 6
## Submission Deadline: 15th March, 2022

#### Reinforcement Learning and DecisionMaking Under Uncertainty IN-STK 5100/9100, Spring 2022, UiO
#### A. George, 1st March, 2022

## Description:
In this exercise, we will implement Policy Iteration and Value Iteration. \
Remember to **only fill in the marked gaps** and do not modify any other bits of code (except trying different parameters)!\
There are experiments provided at the end with which you can **verify that your code is running correctly**.

Let's start!

#### Some Notation:
- $\gamma$ discount factor
- $p(s',r | s,a)$ transition function, i.e., probability of ending up in state $s'$ with reward $r$ when executing action $a$ in state $s$.
- $V(s)$ the estimated value of a state $s$
- $Q(s,a)$ the estimated value of a state $s$ and action $a$

In [95]:
import numpy as np
import random
import matplotlib.pyplot as plt
import math
from itertools import product

## 1) Action Values and Greedy Actions

#### Find the state-action-value for a given state $s$ and action $a$ based on (given) state-values $V$:
$Q(s,a) = \sum_{s',r} p(s',r|s,a)[r+\gamma V(s')]$. 
- **s,a**: both int, the state action pair (indices) to evaluate
- **p**: function taking 4 arguments, the transition function called as p(s_prime, r, s, a), where s_prime and s are states (indices), r is a reward value and a is an action (index)
- **rewards**: array, the possible reward values
- **gamma**: float, discount factor
- **V**: array, state-values for all states

In [96]:
def state_action_value(s,a,p,rewards,gamma,V):
    n_states = np.shape(V)[0]       # the number of states
    Q = 0
    ### FILL IN THE GAP HERE:
    #   - find the state action value based on state values V as indicated in the description
    for s_prime,Vs_prime in enumerate(V):
        for r in rewards:
            Q += p(s_prime, r, s, a) * (r + gamma * Vs_prime)
            
    return Q

#### Find a greedy action and it's value for a given state $s$ based on (given) state-values $V$:
Greedy action = $argmax_{a} \sum_{s',r} p(s',r|s,a)[r+\gamma V(s')]$ and
value of action = $max_{a} \sum_{s',r} p(s',r|s,a)[r+\gamma V(s')]$.
- **s**: int, the state (index) for which we find a greedy action
- **p**: function taking 4 arguments, the transition function called as p(s_prime, r, s, a), where s_prime and s are states (indices), r is a reward value and a is an action (index)
- **rewards**: array, the possible reward values
- **gamma**: float, discount factor
- **n_actions**: int, the number of actions
- **V**: array, state-values for all states

In [97]:
def greedy_action(s, p, rewards, gamma, n_actions, V):
    action = 0
    value = 0
    ### FILL IN THE GAP HERE:
    #   - find a greedy action and it's value based on state values V as indicated in the description
    Q = np.zeros(n_actions)
    for a in range(n_actions):
        Q[a] = state_action_value(s, a, p, rewards, gamma, V)

    action = np.argmax(Q)
    value = Q[action]
    
    return action, value

## 2) Policy Evaluation

#### Find (approximate) the state value function for a given policy $\pi$:
Initialise the state values to be 0, i.e., $V(s)=0$ for all states $s$.  
Update the state values (in place) by using the Bellman Equation: $V(s) = \sum_{a} \pi(a|s) \sum_{s',r} p(s',r|s,a)[r+\gamma V(s')]$.  
Update the state values until for none of the states the change in value is $\geq \theta$.
- **theta**: float, the termination criterion
- **gamma**: float, discount factor
- **rewards**: array, the possible reward values
- **p**: function taking 4 arguments, the transition function called as p(s_prime, r, s, a), where s_prime and s are states (indices), r is a reward value and a is an action (index)
- **pi**: $|\mathcal{S}|\times|\mathcal{A}|$-matrix, the policy, i.e., pi[s][a] is the probability of performing action a in state s.

In [98]:
def policy_evaluation(theta, gamma, rewards, p, pi):
    n_states = np.shape(pi)[0]      # the number of states
    n_actions = np.shape(pi)[1]     # number of actions
    V = np.zeros(n_states)          # the initial state values set to zero
    ### FILL IN THE GAP HERE:
    #   - implement a while loop with condition that the change in values "delta" is greater or equal theta
    #   - within the while loop update the state values and delta
    delta = theta
    while delta >= theta:
        V_new = np.zeros(n_states)

        for s in range(n_states):
            for a in range(n_actions):
               V_new[s] = pi[s,a] * state_action_value(s, a, p, rewards, gamma, V)

        delta = np.max(np.abs(V-V_new))
        V = np.copy(V_new)
    return V

## 3) Policy Iteration

#### Find an optimal (deterministic) policy and the optimal state values by Policy Iteration:
Initialise the policy to the uniform policy, i.e., the policy that selects actions uniformly at random.  
Iteratively, evaluate the policy by calling policy_evaluation() and find a greedy policy for the new values using greedy_action().  
Stop, once the policy is stable, i.e., does not change anymore.
- **theta**: float, the termination criterion
- **gamma**: float, discount factor
- **rewards**: array, the possible reward values
- **p**: a function taking 4 arguments, the transition function called as p(s_prime, r, s, a), where s_prime and s are states (indices), r is a reward value and a is an action (index)
- **n_actions**: int, the number of actions
- **n_states**: int, the number of actions

In [99]:
def policy_iteration(theta, gamma, rewards, p, n_actions, n_states):
    pi = np.zeros((n_states,n_actions))   # the policy
    V = np.zeros(n_states)                # the state values
    ### FILL IN THE GAP HERE:
    #   - initialise a policy pi (matrix) to be the uniform policy
    #   - implement a while loop with condition that the current policy is not stable
    #   - within the while loop 
    #             - find the state values for the current policy using policy_evaluation()
    #             - find a greedy action for every state using greedy_action() and update the policy accordingly
    #             - update whether the policy is stable
    pi = np.ones((n_states,n_actions))/n_actions
    
    stable = False
    while stable is not True:
        pi_prime = np.zeros((n_states,n_actions))
        V_current = policy_evaluation(theta, gamma, rewards, p, pi)

        for s in range(n_states):
            action,value = greedy_action(s, p, rewards, gamma, n_actions, V)

            if value > V_current[s]:
                pi_prime[s,action] = 1
                V[s] = value            
        
        if np.array_equal(pi, pi_prime):
            stable = True
        
        pi = np.copy(pi_prime)

    return pi, V        

## 4) Value Iteration

#### Find an optimal (deterministic) policy and the optimal state values by Value Iteration:
Initialise the state values to be 0, i.e., $V(s)=0$ for all states $s$.  
Update the state values by using the Bellman Optimality Equation: $V(s) = \max_{a} \sum_{s',r} p(s',r|s,a)[r+\gamma V(s')]$.  
Update the state values until for none of the states the change in value is $\geq \theta$.  
Find a greedy action for all states based on the final state values using greedy_action().
- **theta**: float, the termination criterion
- **gamma**: float, discount factor
- **rewards**: array, the possible reward values
- **p**: a function taking 4 arguments, the transition function called as p(s_prime, r, s, a), where s_prime and s are states (indices), r is a reward value and a is an action (index)
- **n_actions**: int, the number of actions
- **n_states**: int, the number of actions

In [100]:
def value_iteration(theta, gamma, rewards, p, n_actions, n_states):
    pi = np.zeros((n_states,n_actions))   # the policy
    V = np.zeros(n_states)                # the state values
    ### FILL IN THE GAP HERE:
    #   - initialise the state values to be zero for all states (already done)
    #   - implement a while loop with condition that the change in values "delta" is greater or equal theta
    #   - within the while loop update the state values and delta  
    #   - find a greedy action for every state and update the policy accordingly
    delta = theta
    while delta >= theta:
        V_new = np.zeros(n_states)

        for s in range(n_states):
            for a in range(n_actions):
                Q = state_action_value(s,a,p,rewards,gamma,V)
                V_new[s] = max(V_new[s],Q)
            
        delta = np.max(np.abs(V-V_new))
        V = np.copy(V_new)

    for s in range(n_states):
        action, value = greedy_action(s, p, rewards, gamma, n_actions, V)
        pi[s, action] = value

    return pi, V     

## 5) Evaluation: Grid World

### Set up the grid world

We consider the following (hard-coded) grid world MDP in which the transitions are deterministic (see example on p.60 & p.65 in the Sutton&Barto book / lecture slides for more details).  

|    |    |    |    |    |
|----|----|----|----|----|
|  x |  A |  x |  B |  x |
|  x |  x |  x |  x |  x |
|  x |  x |  x |  B'|  x |
|  x |  x |  x |  x |  x |
|  x |  A'|  x |  x |  x |

- **States** are the fields marked with x or A,B,A',B'
- **Actions** are to move right, down, left, up
- **Transitions**
    - The next state will be the one following the move, except 
        - when bumping into the boundary, stay in the same state
        - when in A, any action takes you to A'
        - when in B, any action takes you to B'
    - The reward after executing an action is 0, except
        - when bumping into the boundary, the reward is -1
        - when going from A to A', the reward is +10
        - when going from B to B', the reward is +5
- **Discount Factor** $\gamma = 0.9$


In [101]:
gamma = 0.9              # discount factor
rewards = [-1, 0, 5, 10] # list of possible rewards
n_states = 25            # number of states 
n_actions = 4            # number of actions

For convenience, we number the actions as $0 = move\, right$, $1 = move\, down$, $2 = move\, left$, $3 = move\, up$.  
We also number the states as follows:

|    |    |    |    |    |
|----|----|----|----|----|
|  0 |  1 |  2 |  3 |  4 |
|  5 |  6 |  7 |  8 |  9 |
| 10 | 11 | 12 | 13 | 14 |
| 15 | 16 | 17 | 18 | 19 |
| 20 | 21 | 22 | 23 | 24 |

Thus, state A=1, A'=21,B=3,B'=13.

The transitions can be hard-coded as follows:

In [102]:
def p(sp,r,s,a): # output the probability of ending up in state sp with reward r when performing action a in state s
    if s == 1:
        if sp == 21 and r == 10:  # move from field A to A'
            return 1
        else:
            return 0
    if s == 3:
        if sp == 13 and r == 5:   # move from field B to B'
            return 1
        else:
            return 0
    column_s = s % 5
    row_s = (s-column_s)/5
    if row_s == 0 and s == sp and r == -1 and a == 3:     # bump into the upper boundary
        return 1
    if row_s == 4 and s == sp and r == -1 and a == 1:     # bump into the lower boundary
        return 1
    if column_s == 0 and s == sp and r == -1 and a == 2:  # bump into the left boundary
        return 1
    if column_s == 4 and s == sp and r == -1 and a == 0:  # bump into the right boundary
        return 1
    column_sp = sp % 5
    row_sp = (sp-column_sp)/5
    if column_s+1 == column_sp and row_s == row_sp and r == 0 and a == 0:   # move to the right
        return 1
    if column_s-1 == column_sp and row_s == row_sp and r == 0 and a == 2:   # move to the left
        return 1
    if column_s == column_sp and row_s+1 == row_sp and r == 0 and a == 1:   # move down
        return 1
    if column_s == column_sp and row_s-1 == row_sp and r == 0 and a == 3:   # move up
        return 1
    return 0 # all other scenarious are not feasible, i.e., have probability 0
    

### Run Policy & Value Iteration

For both methods we need to set a termination criterium, that determines how accurately we want to approximate the state value function of a policy / the optimal state value function.

In [103]:
theta = 0.001           # value estimation accuracy / termination criterion

To visualise a (deterministic) policy we can call the following function:

In [104]:
def print_policy(pi):
    n_states = np.shape(pi)[0]      # the number of states
    n_actions = np.shape(pi)[1]     # number of actions
    actions = np.argmax(pi, axis=1)
    unicodes =[]
    for s in range(n_states):
        if actions[s] == 0:
            unicodes.append(u"\u2192")
        if actions[s] == 1:
            unicodes.append(u"\u2193")
        if actions[s] == 2:
            unicodes.append(u"\u2190")
        if actions[s] == 3:
            unicodes.append(u"\u2191")
    print("+" + "---+"*5)
    for s in range(0,n_states, 5):
        print(("|" + " {} |"*5).format(*[unicodes[x] for x in range(s, s + 5)]))
        print("+" + "---+"*5)

We can now consider the output of Policy Iteration...

In [105]:
pi, V = policy_iteration(theta, gamma, rewards, p, n_actions, n_states)
V_grid = np.reshape(V, (5, 5))
V_grid = np.around(V_grid, decimals=1, out=None)
print('Optimal state values found by Policy Iteration (rounded):')
print(V_grid)
print('Optimal policy found by Policy Iteration:')
print_policy(pi) 

Optimal state values found by Policy Iteration (rounded):
[[ 9.  15.9 14.3 10.9  9.8]
 [ 8.1 14.3 12.9 11.6 10.4]
 [ 7.3 12.9 11.6 10.4  9.4]
 [ 6.6 11.6 10.4  9.4  8.5]
 [ 5.9 10.4  9.4  8.5  7.6]]
Optimal policy found by Policy Iteration:
+---+---+---+---+---+
| → | → | ← | → | ← |
+---+---+---+---+---+
| → | ↑ | ← | ← | ← |
+---+---+---+---+---+
| → | ↑ | ← | ← | ← |
+---+---+---+---+---+
| → | ↑ | ← | ← | ← |
+---+---+---+---+---+
| → | ↑ | ← | ← | ← |
+---+---+---+---+---+


... and Value Iteration

In [106]:
pi, V = value_iteration(theta, gamma, rewards, p, n_actions, n_states)
V_grid = np.reshape(V, (5, 5))
V_grid = np.around(V_grid, decimals=1, out=None)
print('Optimal state values found by Value Iteration (rounded):')
print(V_grid)
print('Optimal policy found by Value Iteration:')
print_policy(pi) 

Optimal state values found by Value Iteration (rounded):
[[22.  24.4 22.  19.4 17.5]
 [19.8 22.  19.8 17.8 16. ]
 [17.8 19.8 17.8 16.  14.4]
 [16.  17.8 16.  14.4 13. ]
 [14.4 16.  14.4 13.  11.7]]
Optimal policy found by Value Iteration:
+---+---+---+---+---+
| → | → | ← | → | ← |
+---+---+---+---+---+
| → | ↑ | ← | ← | ← |
+---+---+---+---+---+
| → | ↑ | ← | ← | ← |
+---+---+---+---+---+
| → | ↑ | ← | ← | ← |
+---+---+---+---+---+
| → | ↑ | ← | ← | ← |
+---+---+---+---+---+


### Verify your result:
Both Policy Iteration and Value Iteration should give the same results. You can find the optimal state values and a (non-deterministic) optimal policy on p.65 of the book from Sutton&Barto: http://www.incompleteideas.net/book/RLbook2018.pdf.

* My state values are equal for the value iteration, but for policy iterations the values are different. I guess this is due to the stopping criterion is $\pi = \pi'$, and not $\theta > \max_{s\in \mathcal{S}} |v_{k+1}(s)-v_k(s)|$. The optimal actions are however equal.

## 6) Optional (not graded): Implement your own grid world

If you like, you can implement your own grid world.  
- What is your intuition for an optimal policy / optimal state values for this grid world?  
- Apply Policy Iteration and/or Value Iteration to find an optimal policy / optimal state values.
- Do the results match your initial intuition? Why / why not?